In [0]:
!pip3 install spacy_udpipe
!pip3 install wiki_ru_wordnet
!pip3 install pymorphy2
!pip3 install yargy

In [0]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import dictionary, gte, lte, gram, custom
from yargy.pipelines import morph_pipeline

In [0]:
from wiki_ru_wordnet import WikiWordnet
from collections import defaultdict
from google.colab import files
import spacy_udpipe
import ufal.udpipe
import pymorphy2
import string
import re

morph = pymorphy2.MorphAnalyzer()
wikiwordnet = WikiWordnet()

In [4]:
spacy_udpipe.download("ru")
nlp = spacy_udpipe.load("ru")

Already downloaded a model for the 'ru' language


In [6]:
uploaded = files.upload()

Saving decript_clean.txt to decript_clean (1).txt


In [0]:
texts = uploaded['decript_clean.txt'].decode('UTF-8')

In [0]:
texts = texts.split('\n')

**Алгоритм**

1. Проходимся по текстам описаний
2. Каждый делим на предложения 
3. Ищем дату
4. Если есть --> находим глагол
5. Если глагол в словаре --> смотрим на зависимые
6. Если в зависимых библиотека - отлично 
7. Если нет --> ищем с помощью yargy сочетания СУЩ В ИМ + библиотека 
8. Если сущ в зависимых - отлично 


1.   Дата открытия библиотеки – 1916 год +
2.   Дата основания – +
3.   Год открытия – 2001. +
4.   Образована в 1964 г. +
5.   Дата основания: конец 1919 г. – начало 1920 г. +
6. берет свое начало -
7. Официальная дата рождения областной специальной библиотеки для слепых – 5 сентября 1955 года. +
8. Центральная Межпоселенческая библиотека – самая крупная библиотека Сосновского района, дата создания – 1935 год. +



In [0]:
Date = fact('Date', ['day', 'month', 'year'])

MONTHS = {
    'январь', 'февраль', 'март',
    'апрель', 'мая', 'июнь',
    'июль', 'август', 'сентябрь',
    'октябрь', 'ноябрь', 'декабрь'}


MONTH_NAME = dictionary(MONTHS)
DAY = and_(gte(1), lte(31))
YEAR = and_(gte(1900), lte(2100))


a = rule(
    dictionary({'дата', 'год'}),
    gram('NPRO').optional(), 
    dictionary({'основания', 'открытия', 'рождения', 'создания', 'образования'})
)


b = rule(
    gram('ADJF').optional().repeatable(),
    dictionary({'библиотеки'}).optional(),
    gram('PREP').optional(),
    or_(gram('NOUN'), gram('ADJF')).optional()
)

c = dictionary({'–', ':'})
d = dictionary({'конец', 'середина', 'начало'}).optional()


DATE1 = rule(
    
    DAY.interpretation(
        Date.day).optional(),
    
    MONTH_NAME.interpretation(
        Date.month).optional(),
    
    YEAR.interpretation(Date.year)
    
).interpretation(Date)


# Если объединить, получится не очень 

DATE2 = rule(
    
    a, b, c, d,
    
    DAY.interpretation(Date.day).optional(),
    
    MONTH_NAME.interpretation(
        Date.month.normalized()).optional(),
    
    YEAR.interpretation(Date.year)
           
    ).interpretation(Date)


open_date_parser1 = Parser(DATE1)
open_date_parser2 = Parser(DATE2)

In [0]:
open_date_verb = [
                  'возникла', 'возобновилась', 'доступна',
                  'началась', 'образована', 
                  'организована', 'основана', 'открылась', 
                  'открыта', 'появилась', 'учреждена'
]

In [0]:
texts = ['Левковская сельская библиотека основана в 1968 году.', 
         'Петровская сельская библиотека была открыта в 1931 году.', 
         'Центральная детская библиотека была открыта 1 июля 1950 года.',
         'В 1974 году открылась библиотека.']

In [0]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [21]:
open_date = []
log = []
count = 0

length = len(texts)
out = display(progress(0, length), display_id=True)

for index, text in enumerate(texts):
  doc = nlp(text)

  ans = ''

  for sent in doc.sents:

    match = list(open_date_parser1.findall(sent.text))
      
    if match != [] and ans == '':
      mat = list(open_date_parser2.findall(sent.text))
      
      # берем первое вхождение, потому что дата открытия скорее будет в начале текста

      if mat != []:
        ans = ' '.join([i for i in mat[0].fact if i is not None])
        count += 1
      elif str(sent.root).lower() in open_date_verb:
        ans = ' '.join([i for i in match[0].fact if i is not None])
        count += 1
   
  if ans == '':
    log.append(index)

  open_date.append(ans)
  out.update(progress(index, length))
  
      

In [22]:
count

3801

In [0]:
Часто встречается "изба-читальня"

In [0]:
with open('open_dates.txt', 'w', encoding='utf-8') as f:
  for i in open_date:
    f.write(i + '\n')

In [0]:
files.download('submission111111.csv')

In [0]:
files.download('open_dates.txt') 

In [0]:
with open('log_open_dates.txt', 'w', encoding='utf-8') as f:
  for i in log:
    f.write(str(i) + '\n')

files.download('log_open_dates.txt') 